In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
#import geopandas
import pandas as pd

In [2]:
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [3]:
catchment_project='D:\\Xike\\wt\\wt_ReefHydro_all_v411_SC442.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [4]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "D:\Xike\wt\wt_ReefHydro_all_v411_SC442.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\PestFileProductionTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\RebuildInternalSubcatchmentRaster.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\SneakyLinkModelEditor.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\ReefHydroCalModels.dll

[0] Plugins loaded (5/5)

[0] Opening project file: D:\Xike\wt\wt_ReefHydro_all_v411_SC442.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />


In [24]:
#%matplotlib inline
vs = [veneer.Veneer(port=p) for p in ports]
v = vs[0]
the_network = v.network()


allCatchs = the_network['features'].find_by_feature_type('catchment')
#print(len(allCatchs))

#This dictionary object will hopefully hold everything we need
cols = ['SubCat', 'AreaM2', 'Link', 'LinkUpNode', 'LinkDownNode']
networkDetails = pd.DataFrame(columns=cols)#index=len(allCatchs),

intCount = 0
for thisCat in allCatchs:
    
    catName = thisCat['properties']['name']
    catArea = thisCat['properties']['areaInSquareMeters']
    
    linkID = thisCat['properties']['link']
    thisLink = the_network['features'].find_by_id(linkID)
    linkName = thisLink[0]['properties']['name']
    
    fNodeID = thisLink[0]['properties']['from_node']
    tNodeID = thisLink[0]['properties']['to_node']
    
    fNode = the_network['features'].find_by_id(fNodeID)
    tNode = the_network['features'].find_by_id(tNodeID)
    
    fNodeName = fNode[0]['properties']['name']
    tNodeName = tNode[0]['properties']['name']
    
    networkDetails.loc[intCount] = [catName, catArea, linkName, fNodeName, tNodeName]
    #newRow = [catName, catArea, linkName, fNodeName, tNodeName]
    #networkDetails.append([catName, catArea, linkName, fNodeName, tNodeName])
    
    intCount += 1





In [25]:
networkDetails

,SubCat,AreaM2,Link,LinkUpNode,LinkDownNode
0,SC #155,70969500.0,link for catchment SC #155,Node on catchment SC #155,LI02
1,SC #156,80861400.0,link for catchment SC #156,Node on catchment SC #156,Node on catchment SC #155
2,SC #157,71006400.0,link for catchment SC #157,112102A,Node on catchment SC #155
3,SC #487,51161400.0,link for catchment SC #487,Node on catchment SC #487,LI01
4,SC #442,26785800.0,link for catchment SC #442,LI01,112102A


In [26]:
networkDetails.to_csv('D:/aa/WTNetwork.csv', index=False)

In [27]:
kill_all_now(processes)